In [1]:
%matplotlib inline

# SQL Alchemy
from sqlalchemy import create_engine

# PyMySQL 
import pymysql
pymysql.install_as_MySQLdb()

# Config variables
from config import remote_db_endpoint, remote_db_port
from config import remote_dccrime_dbname, remote_dccrime_dbuser, remote_dccrime_dbpwd

import pandas as pd
import fiona
#!pip install geopandas
import geopandas as gpd
import json

In [2]:
#merge_table_final = pd.read_json("geo_table.json")


# Load the first sheet of the JSON file into a data frame
df = pd.read_json("geo_table.json", orient='columns')
#jdata = json.loads()
#df = pd.DataFrame(jdata)
# View the first ten rows
df.head(10)
df.head()
#type(df)



,ADW,Homicide,Robbery,Sex_Abuse,BG,BG_ID,GEOID,total_pop,pct_poverty,pct_unemployed,pct_vacancy,Total_Violent_Crime
0,3,0,2,1,1001,1001,110010001001,1382,0.000000,0.000000,16.620879,6
1,3,0,10,2,1002,1002,110010001002,1463,3.075871,5.433746,15.702479,15
10,2,0,0,0,3002,3002,110010003002,1492,5.093834,2.756892,0.000000,2
100,6,0,12,1,21025,21025,110010021025,502,20.119522,2.769231,0.000000,19
101,12,0,14,1,21026,21026,110010021026,701,15.121255,6.621005,12.054795,27


In [ ]:
#df.to_json("geo_merge_table1.json")

In [3]:
block_groups = gpd.read_file("Census_Block_Groups__2010.geojson")
#block_groups = pd.read_json("Census_Block_Groups__2010.geojson")
block_groups.head()
#block_groups['BG_ID'] = 'A' + block_groups["GEOID"].str[5:]

#block_groups.head()
#type(block_groups)
#block_groups.to_json("geo_merge_table2.json")

,OBJECTID,TRACT,BLKGRP,GEOID,P0010001,P0010002,P0010003,P0010004,P0010005,P0010006,...,OP000013,OP000014,OP000015,OP000016,H0010001,H0010002,H0010003,SHAPE_Length,SHAPE_Area,geometry
0,1,000100,4,110010001004,1000,985,893,34,2,45,...,3,3,4,0,634,533,101,6569.746991,1.162392e+06,POLYGON ((-77.05707530112326 38.90769978737089...
1,2,000201,1,110010002011,3916,3736,2940,259,7,482,...,40,16,114,6,2,2,0,3249.765438,5.395519e+05,POLYGON ((-77.07411930810323 38.91259278700532...
2,3,000202,1,110010002021,1196,1177,1081,25,2,56,...,2,1,6,3,604,547,57,2684.900636,1.920121e+05,POLYGON ((-77.06567930445408 38.91182678677541...
3,4,000202,2,110010002022,813,792,749,3,1,35,...,2,2,6,0,395,366,29,1505.972723,1.355407e+05,POLYGON ((-77.06890030580993 38.90774378687865...
4,5,000202,3,110010002023,1053,1032,954,15,1,57,...,5,4,8,1,505,455,50,1615.539725,1.544660e+05,POLYGON ((-77.06701530462304 38.90509378547841...


In [10]:
#block_groups = block_groups.merge(df, on='GEOID')
block_groups['GEOID'] = block_groups['GEOID'].astype('int64')
block_groups.dtypes
type(block_groups)

geopandas.geodataframe.GeoDataFrame

In [11]:
block_groups = block_groups.merge(df, on='GEOID')

In [12]:
block_groups.head()

,OBJECTID,TRACT,BLKGRP,GEOID,P0010001,P0010002,P0010003,P0010004,P0010005,P0010006,...,Homicide,Robbery,Sex_Abuse,BG,BG_ID,total_pop,pct_poverty,pct_unemployed,pct_vacancy,Total_Violent_Crime
0,1,000100,4,110010001004,1000,985,893,34,2,45,...,0,36,7,1004,1004,1188,8.922559,3.095975,18.136439,74
1,2,000201,1,110010002011,3916,3736,2940,259,7,482,...,0,5,4,2011,2011,3733,1.473346,6.604938,0.000000,10
2,3,000202,1,110010002021,1196,1177,1081,25,2,56,...,1,6,6,2021,2021,1224,15.686275,2.124431,16.824197,20
3,4,000202,2,110010002022,813,792,749,3,1,35,...,1,15,1,2022,2022,566,7.243816,0.000000,6.329114,18
4,5,000202,3,110010002023,1053,1032,954,15,1,57,...,0,23,9,2023,2023,895,20.782123,10.467290,5.135135,51


In [13]:
type(block_groups)

geopandas.geodataframe.GeoDataFrame

In [14]:
block_groups.to_file("bg_test.geojson", driver='GeoJSON')

In [ ]:
geo_merge_table = pd.merge(df, block_groups, left_on='BG_ID2', right_on='BG_ID', how="outer")
geo_merge_table.tail()

In [ ]:
#geo_merge_table.to_json("geo_merge_table.json")
geo_merge_table = geo_merge_table[pd.notnull(geo_merge_table['BG'])]
geo_merge_table.head()
#geo_merge_table.to_json("geo_merge_table.json")
#geo_merge_table.to_file("geo_merge_table.shp")

In [ ]:
type(geo_merge_table)
output1 = geo_merge_table.to_csv("geo_merge_table_test.csv")

In [ ]:
url = "https://opendata.arcgis.com/datasets/c143846b7bf4438c954c5bb28e5d1a21_2.geojson"
df_shapes = gpd.read_file(url)

In [ ]:
df_shapes.head()

In [ ]:
type(df_shapes)